In [ ]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()
output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./menu.xlsx',
                       header=0,
                       dtype=str).fillna('')

print('总数量：' + str(len(input_)))
print()

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for a in range(len(input_)):
    work.put_nowait(a)
    
# = = = = = = = = = = = = = = =

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        a = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for b in range(666):
            try:
                resp = requests.get('https://parts.gmparts.com/wcs/resources/store/11705/productSearch/' + input_.loc[a, 'Url_1'] + '?brand=GM&catalogId=3074457345616677718',
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                json_data = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                src = json_data['catalogEntry']['UserData'][0]['images']['mainImage'].strip()
                
                # = = = = = = = = = = = = = = =
                
                dict_packaging = json_data['catalogEntry']['UserData'][0]['PackagingInformation'] if 'PackagingInformation' in json_data['catalogEntry']['UserData'][0] else {}

                # = = = = = = = = = = = = = = =
                
                manufacturer = json_data['catalogEntry']['manufacturer'].strip()

                # = = = = = = = = = = = = = = =
                
                manufacturer_part_number = json_data['catalogEntry']['manufacturerPartNumber'].strip() if 'manufacturerPartNumber' in json_data['catalogEntry'] else ''

                # = = = = = = = = = = = = = = =
                
                title = json_data['catalogEntry']['name'].strip()

                # = = = = = = = = = = = = = = =
                
                part_number = json_data['catalogEntry']['partNumber'].strip()
                    
                # = = = = = = = = = = = = = = =
                
                list_attribute = json_data['catalogEntry']['attributes']

                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'序号': int(input_.loc[a, 'No.']),
                                         'Part_Number': part_number,
                                         'Manufacturer': manufacturer,
                                         'Manufacturer_Part_Number': manufacturer_part_number,
                                         'Title': title,
                                         'Pic': '',
                                         'Url': 'https://parts.gmparts.com/product/' + input_.loc[a, 'Url_1'],
                                         'Src': src}])

                for attribute in list_attribute:
                    df_temp.loc[0, attribute['name'].strip()] = attribute['value'].strip() + ' ' + attribute['unitOfMeasure'].strip() if 'unitOfMeasure' in attribute else attribute['value'].strip()

                for key in dict_packaging:
                    for key_2 in dict_packaging[key]:
                        if key_2 == 'packageUOM' or key_2 == 'weightUOM':
                            continue
                        elif key_2 == 'length' or key_2 == 'width' or key_2 == 'height':
                            df_temp.loc[0, key.strip() + '_' + key_2.strip()] = dict_packaging[key][key_2].strip() + ' ' + dict_packaging[key]['packageUOM'].strip()
                        elif key_2 == 'weight':
                            df_temp.loc[0, key.strip() + '_' + key_2.strip()] = dict_packaging[key][key_2].strip() + ' ' + dict_packaging[key]['weightUOM'].strip()
                        else:
                            df_temp.loc[0, key.strip() + '_' + key_2.strip()] = dict_packaging[key][key_2].strip()

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
            except:
                time.sleep(0.3)
                continue
        
        # = = = = = = = = = = = = = = =
        
        if crawler_status == 'error':
            df_temp = pd.DataFrame([{'No.': int(input_.loc[a, 'No.']),
                                     'Url_1': input_.loc[a, 'Url_1']}])

            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')
            
        # = = = = = = = = = = = = = = =
        
        print('[' + crawler_status + '] - ' + input_.loc[a, 'Url_1'] +  '\n[尝试次数：' + str(b+1) + '] - [剩余数量：' + str(work.qsize()) + '] - [当前时间：' + datetime.now().strftime('%H:%M:%S') + ']\n')
        
# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print('输出ing...')
print()
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['序号'], ascending=[True]).reset_index(drop=True)
output_correct.to_excel('./part-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.'], ascending=[True]).reset_index(drop=True)
    output_error.to_excel('./part_error.xlsx', index=False)
    print('爬虫存在error')
    print()
print('Done ~')